# Тестирование кастомного минимизатора градиентного спуска
Время работы: 6:25

In [ ]:
import numpy as np
from skimage import color, data, restoration, img_as_float, io
from scipy.signal import convolve2d as conv2

from skimage.restoration import richardson_lucy
from skimage.measure import compare_psnr
from functools import partial
from scipy.optimize import minimize
from scipy.optimize import OptimizeResult
from tqdm import tqdm_notebook

from util import *
from richardson_lucy import *

In [ ]:
def fun(x):
    return np.sum((x - np.array([1, 2]))**2)

In [ ]:
x0 = np.array([-1, -1])
xx = np.array([1, 2])

In [ ]:
fun(xx), fun(x0)

In [ ]:
def fun2(x):
    return (x[0] - 5)**2 + 2 * (x[1] - 10)**2

In [ ]:
minimize_grad(fun2, np.array([0, 0]), alpha=1, disp=True, maxit=100)

In [ ]:
astro = color.rgb2gray(data.astronaut())
liftingbody = img_as_float(io.imread('liftingbody.png'))
iterations = 30

In [ ]:
sz = 21
deg = 5
points_real = [(deg - 1) * sz // deg - 1, sz // deg, sz - 1, sz - 1]
print(points_real)
#points_real = [sz, 0, sz, sz]
psf_bezier = bezier_psf2(points_real)
plt.imshow(psf_bezier, 'gray')
plt.plot(0, 0, 'ro')
plt.plot(points_real[0], points_real[1], 'ro')
plt.plot(points_real[2], points_real[3], 'ro')
#plt.savefig("pic/curved-psf.png")
plt.show()

In [ ]:
lifting_bezier_blurred = convolve2d(liftingbody, psf_bezier, 'same')
plt.imshow(lifting_bezier_blurred, 'gray')
plt.show()

In [ ]:
s_n, S_find = find_noise(lifting_bezier_blurred)
s_n

In [ ]:
deconv = richardson_lucy_matlab(
    lifting_bezier_blurred,
    psf_bezier,
    iterations=iterations,
    clip=True,
    dampar=0.004,
    useFFT=True)
plt.imshow(deconv['image'], 'gray')
plt.show()

In [ ]:
N, M = lifting_bezier_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100 * np.log(1 + np.abs(np.fft.fftn(lifting_bezier_blurred))))
K_shift = np.fft.fftshift(K)

In [ ]:
def funcToMinimizeCurved(xy, I_blurred, crop=100, *args, **kwargs):
    psf = bezier_psf2(xy, n=100)
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))


# partial(funcToMinimizeCurved, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, 
#         dampar=s_n, useFFT=True)([0, 10, 10, 10])

In [ ]:
def funcToMinimizeCurved2(xy, I_blurred, xy2, crop=100, *args, **kwargs):
    psf = bezier_psf2(np.concatenate((np.array(xy), np.array(xy2))), n=100)
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))

In [ ]:
c, r = np.unravel_index(np.argmin(K_shift, axis=None), K_shift.shape)
#nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
x0_2 = np.array([r - n // 2, c - m // 2])
x0_1 = x0_2 / 3 + np.array([2, -2])
x0 = np.concatenate((x0_1, x0_2))
print(x0_1, x0_2)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(np.clip(np.real(K_shift), -1, 1),
           'gray')  #, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n // 2 + x0_2[1], m // 2 + x0_2[0], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
#plt.savefig('pic/kepstr.png')
plt.show()

In [ ]:
res_my_grad = minimize_grad(partial(funcToMinimizeCurved2, I_blurred=lifting_bezier_blurred, 
                                    xy2=x0_2, iterations=iterations, eps=1e-5, dampar=10*s_n, useFFT=True),
                            x0=x0_1, disp=True, alpha=100)
print(res_my_grad)
#res_my_

In [ ]:
res_my_grad2 = minimize_grad(partial(funcToMinimizeCurved2, I_blurred=lifting_bezier_blurred, xy2=x0_2, 
                                     iterations=iterations, eps=1e-5, dampar=10*s_n, useFFT=True),
                             x0=x0_1, disp=True, alpha=1000)
print(res_my_grad)
#res_my_

In [ ]:
res_my_grad2 = OptimizeResult(x=np.concatenate((res_my_grad['x'], x0_2)))

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, 
                                     dampar=s_n, useFFT=False)
psnr_orig = compare_psnr_crop(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, 
                                     dampar=s_n, useFFT=False)
psnr_init = compare_psnr_crop(liftingbody, deconv_init['image'])

points_found = res_my_grad2['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, 
                                      dampar=s_n, useFFT=False)
psnr_found = compare_psnr_crop(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_my_grad2['x'])])